# Assignment 4
*Author: Logan Reine*

In [1]:
%run MLequations_v2.ipynb

ff = lambda x, y, z = "bits" : print("{} {:.4f} {}".format(x, y, z))

# 1 Calculating Entropy

The figure below shows a set of eight Scrabble pieces.

![Local Image](scrabble.png)

**1. What is the entropy (in bits) of the letters in this set?**

In [2]:
scrabble = pd.DataFrame({
    'Letters':['O', 'X', 'Y', 'M', 'O', 'R', 'O', 'N']
})
ff("Entropy:", H(scrabble))

Entropy: 2.4056 bits


**2. What would be the reduction in entropy (i.e., the information gain) in bits if we split these letters into two sets, one containing the vowels and the other containing the consonants?**

In [3]:
vows = pd.DataFrame({
    'Vowels':['O', 'O', 'O']
})
cons = pd.DataFrame({
    'Consonants':['X', 'M', 'R', 'N', 'Y']
})

r_vow = 3/8 * H(vows)
r_con = 5/8 * H(cons)
t_rem = r_vow + r_con

ff("\nInformation Gain:", H(scrabble) - t_rem)


Information Gain: 0.9544 bits


**3. What is the maximum possible entropy (in bits) for a set of eight Scrabble pieces?**

In [4]:
ran_scrabble_8 = pd.DataFrame({
    'Letters':['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
})
ff("Max Possible Entropy:", H(ran_scrabble_8))

Max Possible Entropy: 3.0000 bits


**4. In general, which is preferable when you are playing Scrabble: a set of letters with high entropy, or a set of letters with low entropy?**

    A higher entropy is preferable. The higher the variability in Scrabble tiles, the wider the range in possible word choices.

# 2 Decision Tree Construction

**1. Using this dataset, construct the decision tree that would be generated by the ID3 algorithm, using entropy-based information gain.** 

In [5]:
# ID 	Good Behavior 	Age Below 30     Drug Dependent 	 Recidivist
rec = pd.read_csv("recidivistADT.tsv", delimiter = '\t')

for x in rec.columns[1:-1]:
    ff(x, IG(rec[x], rec))

Good Behavior 0.0817 bits
Age Below 30 0.4591 bits
Drug Dependent 0.1909 bits


In [6]:
t_30 = rec[rec['Age Below 30'] == True]
t_30 = t_30.drop(columns = ['Age Below 30'])

f_30 = rec[rec['Age Below 30'] == False]
f_30 = f_30.drop(columns = ['Age Below 30'])

print(f_30)
for x in f_30.columns[1:-1]:
    ff(x, IG(f_30[x], f_30))

   ID  Good Behavior  Drug Dependent  Recidivist
1   2          False           False       False
3   4           True           False       False
4   5           True            True        True
5   6           True           False       False
Good Behavior 0.1226 bits
Drug Dependent 0.8113 bits


In [7]:
t_drug = f_30[f_30['Drug Dependent'] == True]
t_drug = t_drug.drop(columns = ['Drug Dependent'])

f_drug = f_30[f_30['Drug Dependent'] == False]
f_drug = f_drug.drop(columns = ['Drug Dependent'])

for x in f_drug.columns[1:-1]:
    ff(x, IG(f_drug[x], f_drug))

Good Behavior 0.0000 bits


In [8]:
print('{:^{width}}'.format('[Age < 30]', width = 60))
print('{:^{width}}'.format('TRUE     FALSE', width = 60))
print('{:^{width}}'.format('/           \\', width = 60))
print('{:^{width}}'.format('/             \\', width = 60))
print('{:>{width}}'.format('[Drug Dependent]', width = 47))
print('{:>{width}}'.format('TRUE     FALSE', width = 46))
print('{:>{width}}'.format('/            \\', width = 46))
print('{:>{width}}'.format('/              \\', width = 47))
print('{:>{width}}'.format('[Good Behavior]', width = 56))
print('{:>{width}}'.format('TRUE     FALSE', width = 56))
print('{:>{width}}'.format('/            \\', width = 56))
print('{:>{width}}'.format('/              \\', width = 57))

                         [Age < 30]                         
                       TRUE     FALSE                       
                       /           \                        
                      /             \                       
                               [Drug Dependent]
                                TRUE     FALSE
                                /            \
                               /              \
                                         [Good Behavior]
                                          TRUE     FALSE
                                          /            \
                                         /              \


**2. What prediction will the decision tree generated in part (a) of this question return for the following query?**

**Good Behavior** = false, **Age Below 30** = false, **Drug Dependent** = true

In [9]:
# Older than 30
print(f_30)

# are Drug Dependent
print('\n',t_drug)

   ID  Good Behavior  Drug Dependent  Recidivist
1   2          False           False       False
3   4           True           False       False
4   5           True            True        True
5   6           True           False       False

    ID  Good Behavior  Recidivist
4   5           True        True


    recidivist = true

**3. What prediction will the decision tree generated in part (a) of this question return for the following query?**

**Good Behavior** = true, **Age Below 30** = true, **Drug Dependent** = false

In [10]:
# Younger than 30
print(t_30)

   ID  Good Behavior  Drug Dependent  Recidivist
0   1          False           False        True
2   3          False           False        True


    recidivist = true

# 3 Information Gain

**1. Calculate the entropy for this dataset.**

In [11]:
census = pd.read_csv("sample_census.tsv", delimiter = '\t')
ff('Census Entropy:', H(census))

Census Entropy: 1.2988 bits


**2. Calculate the Gini index for this dataset.**

In [12]:
ff('Gini Index:', gini(census))

Gini Index: 0.5312 bits


**3. When building a decision tree, the easiest way to handle a continuous feature is to define a threshold around which splits will be made. What would be the optimal threshold to split the continuous Age feature (use information gain based on entropy as the feature selection measure)?**

In [13]:
print(census[['ID', 'Age', 'Annual Income']].sort_values(by = 'Age'))

   ID  Age Annual Income
2   3   18     below 25K
5   6   24     below 25K
3   4   28       25K-50K
4   5   37       25K-50K
0   1   39       25K-50K
7   8   40      over 50K
1   2   50       25K-50K
6   7   52       25K-50K


In [14]:
threshold = [27, 39, 49]

for x in threshold:
    i = 0
    thresh_census = census.sort_values(by = 'Age')
    for y in thresh_census['Age']:
        if thresh_census.at[i, 'Age'] > x:
            thresh_census.at[i, 'Age'] = 1
        else: 
            thresh_census.at[i, 'Age'] = 0
        i += 1
    print('Info Gain > {}: {:.4f} bits'.format(x, IG(thresh_census['Age'], thresh_census)))

Info Gain > 27: 0.8113 bits
Info Gain > 39: 0.3476 bits
Info Gain > 49: 0.2044 bits


**4. Calculate information gain (based on entropy) for the Education, Marital Status, and Occupation features.**

In [15]:
for x in census.columns[2:-1]:
    ff(x, IG(census[x], census))

Education 0.7988 bits
Marital Status 0.5488 bits
Occupation 0.7044 bits


**5. Calculate the information gain ratio (based on entropy) for Education, Marital Status, and Occupation features.**

In [16]:
for x in census.columns[2:-1]:
    ff(x, IG(census[x], census) / H(census[[x]]))

Education 0.5683 bits
Marital Status 0.3904 bits
Occupation 0.3697 bits


**6. Calculate information gain using the Gini index for the Education, Marital Status, and Occupation features.**

In [17]:
for x in census.columns[2:-1]:
    ff(x, gini_IG(census[x], census))

Education 0.2812 bits
Marital Status 0.1771 bits
Occupation 0.2396 bits


# 4 Decision Tree Error Pruning

**Shown in the figure below shows a decision tree for predicting heart disease. The descriptive features in this domain describe whether the patient suffers from chest pain (Chest Pain) and blood pressure (Blood Pressure). The binary target feature is Heart Disease. The table below the diagram lists a pruning set from this domain.**

![Local Image](heart.png)

**Using the pruning set, apply reduced error pruning to the decision tree. Assume that the algorithm is applied in a bottom-up, left-to-right fashion. For each iteration of the algorithm, indicate the subtrees considered as pruning candidates, explain why the algorithm chooses to prune or leave these subtrees in the tree, and illustrate the tree that results from each iteration.**

In [18]:
hdisease = pd.read_csv("h_disease.tsv", delimiter = '\t')
chest_pain_false = hdisease[hdisease['Chest Pain'] == False]
print(chest_pain_false[['Blood Pressure', 'Heart Disease']].sort_values(by = 'Blood Pressure'))
print("\n", hdisease.sort_values(by = 'Chest Pain'))

  Blood Pressure  Heart Disease
0           high          False
4           high          False
2            low          False

    ID  Chest Pain Blood Pressure  Heart Disease
0   1       False           high          False
2   3       False            low          False
4   5       False           high          False
1   2        True            low           True
3   4        True           high           True


    Below is the first subtree to be considered.  The parent node returns false, which is correct for 3/3 cases -- returns false if chest pain is false -- so the error rate is 0.  If blood pressure is low, the right leaf returns false, which is correct in 1/1 cases.  If blood pressure is high, The left leaf returns true. This is correct in 0/2 cases, so the error rate is two. The summed error rate of the children nodes exceeds the error rate of the subtree's root node, so this subtree is pruned.

![Local Image](heart_bp.png)

![Local Image](heart_bp2.png)

        Above is the decision tree after the first iteration of pruning.  The root node returns true for the target feature, which misclassifies the target feature in 3/5 cases -- error rate = 3.  After partitioning the data by 'chest pain', the left leaf returns false, this classifies the target feature correctly in 3/3 cases -- error rate = 0.  By default, in a binary decision structure this would mean the sister node also holds an error rate of 0.  The summed error rate of the child nodes is 0, which is less than the root node with an error rate of 3.  This subtree is not pruned, and this would be the final iteration as the entire tree has now been reviewed.

# 5 Random Forest

In [19]:
hd_risk = pd.read_csv('risk.tsv', delimiter = '\t')
risk_a = pd.read_csv('risk_A.tsv', delimiter = '\t')
risk_b = pd.read_csv('risk_B.tsv', delimiter = '\t')
risk_c = pd.read_csv('risk_C.tsv', delimiter = '\t')

**1. Using these bootstrap samples create the decision trees that will be in the random forest model (use entropy based information gain as the feature selection criterion).**

In [20]:
print('\t- SAMPLE A -\n', risk_a)
print('\n- SAMPLE A INFORMATION GAIN - ')
for x in risk_a.columns[1:-1]:
    ff(x, IG(risk_a[x], risk_a))

	- SAMPLE A -
    ID Exercise Family  Risk
0   1    daily    yes   low
1   2   weekly    yes  high
2   2   weekly    yes  high
3   5   rarely     no  high
4   5   rarely     no  high

- SAMPLE A INFORMATION GAIN - 
Exercise 0.7219 bits
Family 0.1710 bits


    'Exercise' has the highest information gain, and coincidentally distributes the data into pure subsets. Since the partitioned sets are pure, there isn't a need to develop a decision tree any further.
![local image](sample_a.png)

In [21]:
print('\t- SAMPLE B -\n', risk_b)
print('\n- SAMPLE B INFORMATION GAIN - ')
for x in risk_b.columns[1:-1]:
    ff(x, IG(risk_b[x], risk_b))

	- SAMPLE B -
    ID  Smoker  Obese  Risk
0   1   False  False   low
1   2    True  False  high
2   2    True  False  high
3   4    True   True  high
4   5    True   True  high

- SAMPLE B INFORMATION GAIN - 
Smoker 0.7219 bits
Obese 0.1710 bits


    The 'smoker' feature has the highest information gain.  Once more, this discriptive feature splits the remaining data into pure sets, so expanding a decision tree any further isn't necessary.

![Local Image](sample_b.png)

In [22]:
print('\t- SAMPLE C -\n', risk_c)
print('\n- SAMPLE C INFORMATION GAIN - ')
for x in risk_c.columns[1:-1]:
    ff(x, IG(risk_c[x], risk_c))

	- SAMPLE C -
    ID  Obese Family  Risk
0   1  False    yes   low
1   1  False    yes   low
2   2  False    yes  high
3   4   True    yes  high
4   5   True     no  high

- SAMPLE C INFORMATION GAIN - 
Obese 0.4200 bits
Family 0.1710 bits


    'Obese' has the highest information gain.  'True' under the descriptive feature obese splits the data into a pure subset, but 'False' does not.  Normally we would use the next descriptive feature to filter the data further, but moving into another decision node for 'Family' wouldn't provide anymore information. 'Obese' being false also split 'Family' into a pure set.

![Local Image](sample_c.png)

**2. Assuming the random forest model you have created uses majority voting, what prediction will it return for the following query:**

**Exercise=rarely, Smoker=false, Obese=true, Family=yes**

    Sample tree A returns: High
    Sample tree B returns: Low
    Sample tree C returns: High

    The majority vote is High.